# Flir acquisition and visualization

In [1]:
import os
import sys
sys.path.insert(0, os.path.realpath('../'))

In [2]:
import LDAQ

In [3]:
# Create FLIR camera acquisition source:
acq_flir = LDAQ.flir.FLIRThermalCamera(acquisition_name="FLIRCam")

In [5]:
# function to calculate average temperature:
def average_temperature(array):
    """This function calculates the average temperature of the image.
    Returned array can be either (N, 1) or (N, M, K), where N is the number
    of samples. 
    
    If return array is of shape (N, 1), then the virtual channel
    will be treated as a data channel. 
    
    If the returned array is of shape
    (N, M, K), then the virtual channel will be treated as an video channel.
    """
    mean = array.mean(axis=1).mean(axis=1)
    return mean.reshape(-1,1)

# add virtual channel:
acq_flir.add_virtual_channel(
    virtual_channel_name="average_temperature", 
    source_channels=["temperature_field"], 
    function=average_temperature)

{'time': array([0.000e+00, 2.000e-03, 4.000e-03, ..., 4.994e+00, 4.996e+00,
        4.998e+00]),
 'channel_names': ['channel 1', 'channel 2'],
 'data': array([[ 512.,  998.],
        [ 512., 1014.],
        [ 512., 1022.],
        ...,
        [ 512.,  906.],
        [ 512.,  944.],
        [ 512.,  975.]]),
 'sample_rate': 500}

In [ ]:
# Create visualization object:
vis = LDAQ.Visualization(refresh_rate=100)

# add virtual channel to visualization:
vis.add_lines(position=(0,0),   # 1st subplots
              source="FLIRCam", # acquisition source
              channels=[0],     # channel index (from acq_flir.channel_names where data channels are stored)
              t_span=3.0)       # time span of the plot
vis.config_subplot((0, 0), ylim=(20, 40)) # configure subplot

# add thermal image to visualization:
vis.add_image(source="FLIRCam", 
              channel="temperature_field", 
              refresh_rate=200) 

In [6]:
# create Core object:
ldaq = LDAQ.Core(acquisitions=[acq_flir], visualization=vis)
ldaq.run(10.) # run for 10 secs after triggering

+--------+-------------------------------------------------+
| HOTKEY |                   DESCRIPTION                   |
+--------+-------------------------------------------------+
|   s    | Start the measurement manually (ignore trigger) |
+--------+-------------------------------------------------+
|   q    |              Stop the measurement               |
+--------+-------------------------------------------------+

Waiting for trigger...
triggered.
	Recording...Measurement finished.


sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x1539f7e2c20>


In [7]:
# Retrieve the measurement data:
measurement = ldaq.get_measurement_dict()
measurement

{'arduino': {'time': array([0.000e+00, 2.000e-03, 4.000e-03, ..., 4.994e+00, 4.996e+00,
         4.998e+00]),
  'channel_names': ['channel 1', 'channel 2'],
  'data': array([[ 512.,  576.],
         [ 512.,  639.],
         [ 512.,  700.],
         ...,
         [-512.,  384.],
         [-512.,  447.],
         [ 512.,  512.]]),
  'sample_rate': 500}}